---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
#print(states)

In [3]:
# Make todo items
# Read the assignment tips
# Read here https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/soJlqi6IEei04BJ_asBq6g


#'''Returns a DataFrame of towns and the states they are in from the 
#university_towns.txt list. The format of the DataFrame should be:
#DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
#columns=["State", "RegionName"]  )
    
#The following cleaning needs to be done:

#1. For "State", removing characters from "[" to the end.
#2. For "RegionName", when applicable, removing every character from " (" to the end.
#3. Depending on how you read the data, you may need to remove newline character '\n'. '''
def get_list_of_university_towns():
    #initiatlize a dataframe struct
    university_towns = pd.DataFrame( [], columns=["State", "RegionName"]  ) 
    #Open the file & prepare to read line by line
    university_towns_filename = 'university_towns.txt'
    university_towns_file = open(university_towns_filename, "r")
    current_state = 'Alabama'
    
    for line in university_towns_file:
         ## If the data line contains "[edit]", it's a state, cache value & continue
        if "edit" in line:
            current_state = line
        ## ELSE add [CACHE, this value] to dataframe
        else:
            #add a row, strip newline
            university_towns.loc[len(university_towns.index)] = [current_state.rstrip(), line.rstrip()] 
    
    #Cleanse data:
    ## Remove "\[.*"
    university_towns['State'] = university_towns['State'].replace(to_replace='\[.*', value='', regex=True)
    ## Remove "\(.*"
    university_towns['RegionName'] = university_towns['RegionName'].replace(to_replace='\(.*', value='', regex=True)
    ## Remove trailing spacees
    university_towns['RegionName'] = university_towns['RegionName'].str.strip()
    return university_towns


get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


In [4]:
# Make todo items
# Read the assignment tips
# Read here https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/soJlqi6IEei04BJ_asBq6g


'''read in excel, skip the header, skip the first 2 rows, use only 2 cols '''
def get_gdp_dataframe():
    gdp_by_quarter = pd.read_excel(io='gdplev.xls', header=5, skiprows=214, usecols=[4,6])
    gdp_by_quarter.columns = [ "quarter" , "gdp" ]
    return gdp_by_quarter
  
    
'''Must have 2 consecutive declining quarters'''
def is_consecutive_quarter_decline(row):
    return ( row['gdp'] < row['prev_gdp'] and row['prev_gdp'] < row['prev_prev_gdp'] )


'''Returns the year and quarter of the recession start time as a 
string value in a format such as 2005q3'''
def get_recession_start():
    #Load-in gdplev.xls
    gdp_by_quarter = get_gdp_dataframe()
    
    #recession == 2 consecutive quarters GDP DECLINE 
    #create (2) columns for previous quarters (for comparison)
    gdp_by_quarter['prev_gdp'] = gdp_by_quarter['gdp'].shift(1)
    gdp_by_quarter.loc[0, 'prev_gdp'] = gdp_by_quarter.loc[0]['gdp']
    gdp_by_quarter['prev_prev_gdp'] = gdp_by_quarter['prev_gdp'].shift(1)
    gdp_by_quarter.loc[0, 'prev_prev_gdp'] = gdp_by_quarter.loc[0, 'prev_gdp']

    #Now simply create mask where gdp < prev & prev < prev-prev 
    gdp_recession_mask = gdp_by_quarter.apply(is_consecutive_quarter_decline, axis=1)
    gdp_recession_start_quarters = gdp_by_quarter[gdp_recession_mask]
    
    #The recession starts @ first consecutive quarter decline 
    recession_start_idx = (gdp_recession_start_quarters.index[0] - 1) #2 quarters happened, so pick the first
    recession_start = gdp_by_quarter.loc[recession_start_idx, 'quarter']

    return (recession_start)


get_recession_start()

'2008q3'

In [5]:
# Make todo items
# Read the assignment tips
# Read here https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/soJlqi6IEei04BJ_asBq6g


'''Must have 2 consecutive increasing quarters'''
def is_consecutive_quarter_increase(row):
    return ( row['gdp'] > row['prev_gdp'] and row['prev_gdp'] > row['prev_prev_gdp'] )


'''Returns the year and quarter of the recession end time as a 
string value in a format such as 2005q3'''
def get_recession_end():
    #Load-in gdplev.xls
    gdp_by_quarter = get_gdp_dataframe()
    
    #Search for recession end (recession_end == 2 consecutive quarters GDP INCREASE)
    #create (2) columns for previous quarters (for comparison)
    gdp_by_quarter['prev_gdp'] = gdp_by_quarter['gdp'].shift(1)
    gdp_by_quarter.loc[0, 'prev_gdp'] = gdp_by_quarter.loc[0]['gdp']
    gdp_by_quarter['prev_prev_gdp'] = gdp_by_quarter['prev_gdp'].shift(1)
    gdp_by_quarter.loc[0, 'prev_prev_gdp'] = gdp_by_quarter.loc[0, 'prev_gdp']

    #Get recession start & slice data to ONLY have this data
    recession_start = get_recession_start()
    recession_start_row = gdp_by_quarter[gdp_by_quarter['quarter'] == recession_start]
    recession_start_index = recession_start_row.index[0]
    gdp_by_quarter_after_recession_start = \
        gdp_by_quarter.iloc[recession_start_index:(gdp_by_quarter.index[-1] + 1)]

    #Now simply create mask where gdp < prev & prev < prev-prev 
    gdp_recession_mask = gdp_by_quarter_after_recession_start.apply(is_consecutive_quarter_increase, axis=1)
    gd_recession_end_quarters = gdp_by_quarter_after_recession_start[gdp_recession_mask]
    
    #The recession start will be the first occurence of this phenomenom
    recession_end_idx = gd_recession_end_quarters.index[0]
    recession_end = gd_recession_end_quarters.loc[recession_end_idx, 'quarter']

    return (recession_end)

get_recession_end()



'2009q4'

In [6]:
# Make todo items
# Read the assignment tips
# Read here https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/soJlqi6IEei04BJ_asBq6g

'''Returns the year and quarter of the recession bottom time as a 
string value in a format such as 2005q3'''
def get_recession_bottom():
    #Get GDP
    gdp_by_quarter = get_gdp_dataframe()
    #Get the start
    start_quarter = get_recession_start()
    #Get the end
    end_quarter = get_recession_end()
    #Find the slice of rows in between
    recession_start_row = gdp_by_quarter[gdp_by_quarter['quarter'] == start_quarter]
    recession_start_index = recession_start_row.index[0]
    recession_end_row = gdp_by_quarter[gdp_by_quarter['quarter'] == end_quarter]
    recession_end_index = recession_end_row.index[0]
    gdp_by_quarter_recession = gdp_by_quarter.iloc[recession_start_index:\
                                                  recession_end_index]
    gdp_by_quarter_recession = \
        gdp_by_quarter.iloc[recession_start_index:(recession_end_index+1)]
    
    #min_quarter_gdp = gdp_by_quarter_recession[ \
            #gdp_by_quarter_recession.where(\
            #gdp_by_quarter_recession['gdp'] == \
            #(gdp_by_quarter_recession['gdp'].min()))]
    
    min_quarter_gdp = gdp_by_quarter_recession.where(\
            gdp_by_quarter_recession['gdp'] == \
            (gdp_by_quarter_recession['gdp'].min()))
    min_quarter_gdp = min_quarter_gdp.dropna()
    
    return (min_quarter_gdp['quarter'].iloc[0])

get_recession_bottom()

'2009q2'

In [64]:
# Make todo items
# Read the assignment tips
# Read here https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/soJlqi6IEei04BJ_asBq6g

'''Use the state abbreviation map to transfer from abbreviation to full name'''
def replace_state_abbreviation_with_full_name(row):
    row['State'] = states[row['State']]
    return (row)


'''Combine 3 month segments {1:{01-03}, 2:{04-06}, 3:{07-09}, 4:{10-12}}'''
def create_quarters_from_months_row(row):
    month_counter = 0
    quarterly_total = 0
    col_idx = 0
    for month_val in row:
        #update the monthly counter & quarterly total
        month_counter = month_counter + 1
        quarterly_total = quarterly_total + month_val
        
        #Get relevant column info
        year_month = row.index[col_idx]
        year = year_month[0:4]  
        month = year_month[5:7] #skip the '-'
        col_idx = col_idx + 1        
        
        #Handle the completion of quarters
        if month_counter == 3:
            #q1 - add
            row[year+'q1'] = quarterly_total
            #reset total
            quarterly_total = 0
        elif month_counter == 6:
            #q2 - add
            row[year+'q2'] = quarterly_total
            #reset total
            quarterly_total = 0
        elif month_counter == 9:
            #q3 - add
            row[year+'q3'] = quarterly_total
            #reset total
            quarterly_total = 0
        elif month_counter == 12:
            #q4 - add
            row[year+'q4'] = quarterly_total
            #reset total
            quarterly_total = 0
            #reset counter
            month_counter = 0
            
    #Account for non-complete quarters
    if month_counter != 0:
        quarter = 'q1'
        if month_counter > 3 and month_counter <= 6:
            quarter = 'q2'
        elif month_counter > 6 and month_counter <= 9:
            quarter = 'q3'
        elif month_counter > 9 and month_counter <= 12:
            quarter = 'q4'
        else:
            quarter = 'q1'
            
        row[year+quarter] = quarterly_total
        
    return (row)

'''Converts the housing data to quarters and returns it as mean 
values in a dataframe. This dataframe should be a dataframe with
columns for 2000q1 through 2016q3, and should have a multi-index
in the shape of ["State","RegionName"].

Q1 is {01 - 03}, January through March, 
Q2 is {04 - 06}, April through June, 
Q3 is {07 - 09}, July through September, 
Q4 is {10 - 12}, October through December.
    
The resulting dataframe should have 67 columns, and 10,730 rows.
'''
def convert_housing_data_to_quarters():
    #Step 1: read the excel file as a dataframe
    base_city_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    #Map all state acronyms to actual state names
    city_data = base_city_data.apply(replace_state_abbreviation_with_full_name, axis=1)
    
    #Setup multi-index as [State (col 2), RegionName (col 1)]
    city_data = city_data.set_index(['State', 'RegionName']).sort_index()
    
    # Strip useless columns
    city_data = city_data.iloc[:, 49:]

    #Combine months into quarters
    #This method is too slow! city_data = city_data.apply(create_quarters_from_months_col, axis=0)
    
    more_months_exist = True
    month_first = 0
    month_last = 3
    total_months = len(city_data.columns)
    while more_months_exist == True :
        #Account for last quarter potentially having fewer than all 3 months?
        if month_last > total_months:
            month_last = total_months
            
        #Get the data
        quarterly_data = city_data.iloc[:, month_first:month_last]
        
        #Get the quarter
        quarter_value = int( (month_first % 12) / 3) + 1
        quarter = 'q' + str(quarter_value)
          
        #Get the year
        year_month_label = quarterly_data.columns[0]
        year = year_month_label[0:4]
        year_quarter = year + quarter
        
        #Get the mean for this quarter
        city_data[year_quarter] = quarterly_data.mean(axis=1, skipna=True, numeric_only=True)
        
        #update the loop control var
        month_first = month_first + 3
        month_last = month_last + 3
            
        #If we've done every month we are done
        if month_first >= total_months:
            more_months_exist = False
 
    #Strip NON-quarterly columns?
    city_data = city_data.iloc[:, [x for x in range(200, 267, 1)]]
    
    return (city_data)
             
    
#convert_housing_data_to_quarters()
convert_housing_data_to_quarters().loc[[('Ohio','Akron'),('Ohio','Dayton')]].loc[:,['2010q3','2015q2','2016q4']]
#convert_housing_data_to_quarters().loc[[('Ohio','Akron'),('Ohio','Dayton')]]


#Problems:
## Dayton 2010q3 is NaN instead of 57600.0000
### What are Daytons 2010_7: 2010_8: 2010_9:
#### (SOLUTION) need to take the ACTUAL MEAN of 3 columns at a time, NOT simply the average
####    -> some columns (months) may be NaN BUT the mean will take the average of available values for the quarter
####    -> so 2, 3, 4 => 3 BUT NaN, 6, 8 => 7
####    -> need to group 3 columns at a time and apply mean() fcn to them?!
####       -> where mean() is either pre-existing or defined
##### Numbers coordinates for first data: 182 FW


# TO test the output, try the following line:
#convert_housing_data_to_quarters().loc[[('Ohio','Akron'),('Ohio','Dayton')]].loc[:,['2010q3','2015q2','2016q4']]
#
# The result should be:
#State.  RegionName.  2010q3.            2015q2.         2016q4
#Ohio    Akron        67577.66666667     61833.333333    NaN
#Ohio    Dayton       57600.00000000     50000.000000    NaN


/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:910: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_nested_tuple(tup)


2010q3        2015q2  2016q4
State RegionName                                    
Ohio  Akron       67566.666667  61833.333333     NaN
      Dayton      57600.000000  50000.000000     NaN

In [69]:
# Make todo items
# Read the assignment tips
# Read here https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/soJlqi6IEei04BJ_asBq6g



def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    #OVERALL HYPOTHESIS:
    #University towns have their mean housing prices less effected by recessions. 
    #
    #Run a t-test to compare the ratio of the mean price of houses in 
    #  university towns the quarter before the recession starts 
    #  compared to the recession bottom
    
    #1. Cities/regions which are college regions
    university_towns = get_list_of_university_towns()
    
    #2. Cities/regions & avg housing price
    #TODO: BJT - do we need to strip out N/A data for t_test?!
    town_housing_data = convert_housing_data_to_quarters()
    
    #3. Recession data
    #TODO: BJT - need clarification on this, for now just use result of prev. method
    #get the quarter BEFORE the recession? START of recession?
    recession_start_quarter = get_recession_start()
    
    #get the recession bottom
    recession_bottom_quarter = get_recession_bottom()
    
    #Create data showing decline or growth of housing prices 
    #  between the recession start & recession bottom
    town_housing_data['recession_start_to_bottom'] =\
        town_housing_data[recession_start_quarter] /\
        town_housing_data[recession_bottom_quarter]
    
    #get the univeristy town data
    university_housing_data = town_housing_data.merge(\
                                university_towns,\
                                how='inner',\
                                left_index=True,\
                                right_on=['State', 'RegionName'])
    #Setup multi-index as [State (col 2), RegionName (col 1)]
    university_housing_data =\
        university_housing_data.set_index(\
        ['State', 'RegionName']).sort_index()
    
    #get the NON univeristy town data
    non_university_housing_data = town_housing_data[\
                                ~town_housing_data.index.isin(\
                                university_housing_data.index)]
    
    #Run t_test comparing university town values to non-university town values
    #  Figure out t-test stuff
    #    return whether alternative hypothesisis (groups are same) is true
    t_test_results = ttest_ind(\
            non_university_housing_data['recession_start_to_bottom'],\
            university_housing_data['recession_start_to_bottom'],\
            nan_policy='omit')
    
    relevant_non_university_housing_data =\
        non_university_housing_data['recession_start_to_bottom']
    non_university_housing_loss_mean =\
        relevant_non_university_housing_data.mean()
    #print(relevant_non_university_housing_data)
    
    relevant_university_housing_data =\
        university_housing_data['recession_start_to_bottom']
    #print(relevant_university_housing_data)
    university_housing_loss_mean =\
        relevant_university_housing_data.mean()
    
    #return tuple (different, p, better) where
    #  different = True if the t-test is True (p<0.01)
    #  p = scipi.stats.ttest_ind() return value
    #  better = lower mean price ratio? "university town" : "non-university town"
    #  e.g. return (True, 0.001, "university town")/
    #return (relevant_non_university_housing_data.mean(), relevant_university_housing_data.mean())
    different = t_test_results.pvalue < 0.01
    
    better = 'non-university town'
    if university_housing_loss_mean < non_university_housing_loss_mean:
        better = 'university town'
    
    return (different, t_test_results.pvalue, better)

run_ttest()

(True, 0.005496427353633026, 'university town')